In [4]:
import cv2
import numpy as np

# 读取两幅图像
img1 = cv2.imread('image1.jpg', cv2.IMREAD_GRAYSCALE)  # 查询图像
img2 = cv2.imread('image2.jpg', cv2.IMREAD_GRAYSCALE)  # 训练(场景)图像

# 初始化ORB检测器
orb = cv2.ORB_create()

# 使用ORB寻找关键点和描述符
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

# 创建匹配器
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# 匹配描述符
matches = matcher.match(des1, des2)

# 根据距离排序
matches = sorted(matches, key=lambda x: x.distance)

# 提取好的匹配点
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = kp1[match.queryIdx].pt
    points2[i, :] = kp2[match.trainIdx].pt

# 计算单应性矩阵
H, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

# 使用单应性矩阵变换图像
height, width = img2.shape
im1_reg = cv2.warpPerspective(img1, H, (width, height))

def show_resized_image(image, title, scale_percent=50):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    cv2.imshow(title, resized)

# 假设 `im1_reg` 和 `img2` 已经是变换后和参考图像
show_resized_image(im1_reg, 'Registered Image', 25)  # 缩小到原来的50%
show_resized_image(img2, 'Reference Image', 25)      # 缩小到原来的50%

cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np

# 读取立体图像对
imgL = cv2.imread('view5.png', cv2.IMREAD_GRAYSCALE)  # 左图
imgR = cv2.imread('view6.png', cv2.IMREAD_GRAYSCALE) # 右图

# 初始化立体匹配的SGBM模型
window_size = 5
min_disp = 16
num_disp = 112 - min_disp

stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=window_size,
    P1=8 * 3 * window_size ** 2,
    P2=32 * 3 * window_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=15,
    speckleWindowSize=0,
    speckleRange=2,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

# 计算视差图
disparity = stereo.compute(imgL, imgR).astype(np.float32) / 16.0

# 归一化视差图以便显示
disparity_normalized = cv2.normalize(disparity, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

def show_resized_image(image, title, scale_percent=50):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    cv2.imshow(title, resized)

# 显示视差图
show_resized_image(disparity_normalized ,'Disparity',25)
cv2.waitKey(0)
cv2.destroyAllWindows()
